In [ ]:
import sqlite3
import numpy as np
from datetime import datetime, timedelta

conn = sqlite3.connect('testDb.sqlite')
cur = conn.cursor()

Create tables for test:

In [ ]:
cur.executescript("""

Drop Table If Exists Timestamp;
Create Table Timestamp(
    id Integer Primary Key Not Null,
    timestamp timestamp Unique
);

Drop Table If Exists Tag;
Create Table Tag(
    id Integer Primary Key Not Null,
    name Text Unique,
    comment Text,
    unit Text
);

Drop Table If Exists Data;
Create Table Data(
    id Integer Primary Key Not Null,
    value float,
    timestamp_id Integer,
    tag_id Integer,
    Unique(timestamp_id, tag_id)    
);

Drop Table If Exists PreprocessMethod;
Create Table PreprocessMethod(
    id Integer Primary Key Not Null,
    name Text Unique,
    description Text
);

Drop Table If Exists PreprocessedTag;
Create Table PreprocessedTag(
    tag_id Integer Primary Key Not Null,
    preprocess_method_id Integer,
    source_tag_id Integer,
    Unique(preprocess_method_id, source_tag_id)
);

Drop Table If Exists EventTag;
Create Table EventTag(
    tag_id Integer Primary Key Not Null
);

Drop Table If Exists NormalizeParameter;
Create Table NormalizeParameter(
    tag_id Integer Primary Key Not Null,
    mu float,
    sd float
);

""")

* PV[i], i = 0:123
* MV[j], j = 0:12
* EV[k], k = 0:3
* PreprocessMethod[l], l = 0:3
* Timestamp[t], t = 0:1000

In [ ]:
nSample = 2**10
nPv = 123
nMv = 20
nEv = 7
nPreprocess = 3

In [ ]:
t0 = datetime.now()
timestamp = np.array([t0 + timedelta(minutes = 5 *k1) for k1 in range(nSample) ]) # (*,)
dataPv = np.random.randn(nSample, nPv) # (*, nPv)
dataMv = np.random.randn(nSample, nMv) # (*, nMv)
dataEv = np.random.randint(2, size = (nSample, nEv)).astype(np.float) # (*, nEv)
dataPreprocessed = [
    np.random.randn(nSample, nPv)
        for k1 in range(nPreprocess)]

dataPv[np.random.rand(*dataPv.shape) < 0.05] = np.nan
dataMv[np.random.rand(*dataMv.shape) < 0.05] = np.nan
dataEv[np.random.rand(*dataEv.shape) < 0.05] = np.nan
for k1 in range(nPreprocess):
    dataPreprocessed[k1][np.random.rand(*dataPreprocessed[k1].shape) < 0.05] = np.nan

initialize timestamp table:

In [ ]:
for t in range(nSample):
    cur.execute("""
        Insert Or Ignore Into Timestamp (
            timestamp
            ) values
            (
            ?
            )
    """, (timestamp[t],))    

initialize tag table:

In [ ]:
sql = """Insert Or Ignore Into Tag(
            name
        ) values
        (
        ?
        )"""


for k1 in range(nPv):
    cur.execute(sql, ("PV%04d" % (k1+1),))
for k1 in range(nMv):
    cur.execute(sql, ("MV%04d" % (k1+1),))
for k1 in range(nEv):
    cur.execute(sql, ("EV%04d" % (k1+1),))
for k2 in range(nPreprocess):
    for k1 in range(nPv):
        cur.execute(sql, ("Preprocessed%04dPV%04d" % (k2+1, k1+1),))

initialize data table:

In [ ]:
sql1 = """
    Insert Or Ignore Into Data(
        timestamp_id
        , tag_id
        , value
    ) values (
        ?
        ,?
        ,?)
"""

sql2 = """
    Select id From Timestamp where timestamp = ? Limit 1
    """
sql3 = """
    Select id From Tag Where name = ? Limit 1
    """

for (nn, prefix, data) in [
    (nPv, "PV", dataPv)
    , (nMv, "MV", dataMv)
    , (nEv, "EV", dataEv)
    , (nPv, "Preprocessed0001PV", dataPreprocessed[0])
    , (nPv, "Preprocessed0002PV", dataPreprocessed[1])
    , (nPv, "Preprocessed0003PV", dataPreprocessed[2])
    ]:

    for k1 in range(nn):
        for k2 in range(nSample):
            cur.execute(sql2, (timestamp[k2],))
            timestamp_id, = cur.fetchone()

            cur.execute(sql3, (prefix + "%04d" % (k1+1),))
            tag_id, = cur.fetchone()

            val = None if np.isnan(data[k2, k1]) else float(data[k2, k1])

            cur.execute(sql1, (timestamp_id, tag_id, val))

initialize table: PreprocessMethod

In [ ]:
sql = """
Insert or Ignore Into PreprocessMethod(
    name 
    )
    values
    (
    ?)
"""

for k2 in range(nPreprocess):
    cur.execute(sql, ("Preprocess%04d" % (k2+1),))

initialize table: PreprocessedTag

In [ ]:
sql1 = """
    Insert Or Ignore Into PreprocessedTag(
        tag_id
        , preprocess_method_id
        , source_tag_id
        ) 
    values
    (
        ?
        ,?
        ,?
        )"""

sql2 = """
    Select 
        id
        From Tag
        Where name = ?
    """

sql3 = """
    Select
        id
        From PreprocessMethod
        Where name = ?
"""

for k1 in range(nPv):
    for k2 in range(nPreprocess):
        
        source_tag = "PV%04d" % (k1+1)
        preprocessMethod = "Preprocess%04d" % (k2+1)
        tag = "Preprocessed%04d" % (k2+1) + source_tag
        
        cur.execute(sql2, (tag,))
        tag_id, = cur.fetchone()
        cur.execute(sql2, (source_tag,))
        source_tag_id, = cur.fetchone()

        cur.execute(sql3, (preprocessMethod,))
        preprocess_method_id, = cur.fetchone()
        cur.execute(sql1, (tag_id, preprocess_method_id, source_tag_id,))

initialize table: EventTag

In [ ]:
for k1 in range(nEv):

    cur.execute("""
    Select 
        id 
        From Tag
        Where name = ?
    """, ("EV%04d" % (k1+1),))

    tag_id, = cur.fetchone()

    cur.execute("""
    Insert Or Ignore Into EventTag
        (tag_id) 
        values 
        (?)
    """, (tag_id,))

initialize table :NormalizeParameter

In [ ]:
for k1 in range(nPv):

    cur.execute("""
    Select 
        id 
        From Tag
        Where name = ?
    """, ("PV%04d" % (k1+1),))

    tag_id, = cur.fetchone()

    cur.execute("""
    Insert Or Ignore Into NormalizeParameter
        (tag_id, mu, sd) 
        values 
        (?, ?, ?)
    """, (tag_id, *np.random.rand(2)))

In [ ]:
conn.commit()